In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from datetime import date,timedelta,datetime
import re

In [3]:
path = r'./chromedriver'
cService = webdriver.ChromeService(executable_path=r'./chromedriver')
driver = webdriver.Chrome(service = cService)

In [4]:
url = 'https://bitcointalk.org/index.php?board=1.0'
driver.get(url)

In [5]:
current_date = datetime.now()
formatted_date = current_date.strftime('%B %d, %Y')

In [6]:
df = pd.DataFrame(columns=['subject','replies','views','comment','date','user','activity','merit'])

In [7]:
def getComment():
    pagenow = 1
    comment = []
    flag = True
    users = []
    dates = []
    acts = []
    merit = []
    while flag == True:
        dt = driver.find_elements(By.XPATH,"//tbody//tr//td[@valign='middle']//div[@class='smalltext']")
        content = driver.find_elements(By.XPATH,"//td[@class='td_headerandpost']//div[@class='post']")
        user = driver.find_elements(By.XPATH,"//td[@class='poster_info']//b//a")
        act = driver.find_elements(By.XPATH,"//td[@class='poster_info']//div[@class='smalltext']")
        content_list = []
        dt_list = []
        user_list = []
        act_list = []
        for i in range(min(len(content),len(user),len(act))):
            content_list.append(content[i].get_attribute('textContent'))
            user_list.append(user[i].get_attribute('textContent'))
            act_list.append(act[i].get_attribute('textContent'))
        for j in range(len(dt)):
            dt_list.append(dt[j].get_attribute('textContent'))
        indexes_to_remove2 = [index for index, item in enumerate(dt_list) if item.startswith('Merited')]
        for index2 in sorted(indexes_to_remove2, reverse=True):
            del dt_list[index2]
        indexes_to_remove = [index for index, item in enumerate(dt_list) if item[0].isdigit()]
        for index in sorted(indexes_to_remove, reverse=True):
            del content_list[index]
            del dt_list[index]
            del act_list[index]
            del user_list[index]
        
        for z in range(min(len(dt_list),len(user_list),len(content_list),len(act_list))):
            comment.append(content_list[z])
            users.append(user_list[z])
            dates.append(dt_list[z])
            match1 = re.search(r'Activity: (\d+)', act_list[z])
            activity_number = match1.group(1) if match1 else None
            acts.append(activity_number)
            match2 = re.search(r'Merit: (\d+)', act_list[z])
            merit_number = match2.group(1) if match2 else None
            merit.append(merit_number)
        page = driver.find_elements(By.XPATH,"//a[@class='navPages']")
        for k in range(len(page)):
            try:
                value = int(page[k].get_attribute('textContent'))
            except:
                value = 0
            if value == pagenow + 1:
                pagenow = pagenow+1
                page[k].click()
                time.sleep(2)
                break
        else:
            flag = False
    for w in range(len(dates)):
        if dates[w].startswith('T'):
            dates[w]  = formatted_date
        else:
            dates[w] = remove_after_second_comma(dates[w])
    return comment,dates,users,acts,merit

In [8]:
def remove_after_second_comma(s):
    comma_indices = [i for i, char in enumerate(s) if char == ',']
    if len(comma_indices) < 2:
        return s
    else:
        return s[:comma_indices[1]]

In [9]:
pagenow = 1
flag = True
while flag:
    a = driver.find_elements(By.XPATH,"//td[@class='windowbg']//span/a")
    b = driver.find_elements(By.XPATH, "//td[@class='windowbg' and @valign='middle' and @width='4%' and @align='center']")
    c = driver.find_elements(By.XPATH, "//td[contains(@class, 'windowbg2') and contains(@class, 'lastpostcol')]//span[@class='smalltext']")
    url_list = []
    sub_list = []
    rep_list = []
    view_list = []
    for j in range(min(len(a),len(b)//2)):
        url_list.append(a[j].get_attribute('href'))
        sub_list.append(a[j].get_attribute('textContent'))
        v1 = b[2*j].get_attribute('textContent').replace('\n', '').replace('\t', '')
        v2 = b[2*j+1].get_attribute('textContent').replace('\n', '').replace('\t', '')
        rep_list.append(v1)
        view_list.append(v2)
    for z in range(len(url_list)):
        current_url = driver.current_url
        driver.get(url_list[z])
        c,d,u,a,m = getComment()
        for i in range(len(c)):
            ct = [sub_list[z],rep_list[z],view_list[z],c[i],d[i],u[i],a[i],m[i]]
            ctm = pd.DataFrame(ct).T
            ctm.columns=['subject','replies','views','comment','date','user','activity','merit']
            df = pd.concat([df,ctm])
            
        df = df.reset_index(drop=True)
        time.sleep(3)
        driver.get(current_url) 
    
    if pagenow == 1530:
        break
    pages = driver.find_elements(By.XPATH,"//td[@class='middletext']/a[@class='navPages']")
    for k in range(len(pages)):
        try:
            value = int(pages[k].get_attribute('textContent'))
        except:
            value = 0
        if value == pagenow + 1:
            pagenow = pagenow + 1
            time.sleep(5)
            pages[k].click()
            break
    else:
        flag = False  

1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393
1394
1395
1396
1397
1398
1399
1400


In [12]:
dfs = df.dropna()

In [14]:
outPath = "./data/btccomment.csv"
dfs.to_csv(outPath)